<a href="https://colab.research.google.com/github/Juniorffonseca/python-valorant-preditor/blob/main/tcc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fazendo meu projeto do TCC, que foi feito em R, na linguagem Python.
O projeto consiste em usar redes neurais para prever resultado de partidas do cenário profissional de Valorant


In [8]:
# Carregando pacotes
import re
import requests
from bs4 import BeautifulSoup
import itertools
import pandas as pd
import glob
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from google.colab import files
files.upload()

Saving 2023-04-23_partidas.csv to 2023-04-23_partidas.csv
Saving 2023-05-07_partidas.csv to 2023-05-07_partidas.csv
Saving 2023-05-08_partidas.csv to 2023-05-08_partidas.csv
Saving 2023-04-15_partidas.csv to 2023-04-15_partidas.csv
Saving 2023-04-29_partidas.csv to 2023-04-29_partidas.csv
Saving 2023-04-16_partidas.csv to 2023-04-16_partidas.csv
Saving 2023-04-30_partidas.csv to 2023-04-30_partidas.csv
Saving 2023-05-06_partidas.csv to 2023-05-06_partidas.csv
Saving 2023-04-08_partidas.csv to 2023-04-08_partidas.csv
Saving 2023-04-13_partidas.csv to 2023-04-13_partidas.csv
Saving 2023-02-20_partidas.csv to 2023-02-20_partidas.csv
Saving 2023-04-06_partidas.csv to 2023-04-06_partidas.csv
Saving 2023-04-11_partidas.csv to 2023-04-11_partidas.csv
Saving 2023-04-22_partidas.csv to 2023-04-22_partidas.csv
Saving 2023-05-04_partidas.csv to 2023-05-04_partidas.csv
Saving 2023-02-19_partidas.csv to 2023-02-19_partidas.csv
Saving 2023-03-10_partidas.csv to 2023-03-10_partidas.csv
Saving 2023-04

{'2023-04-23_partidas.csv': b'"";"time1RND";"time1R";"time1ACS";"time1KAST";"time1KD";"time1ADR";"time1KPR";"time1APR";"time1FKPR";"time1FDPR";"time1K";"time1D";"time1A";"time1FK";"time1FD";"time2RND";"time2R";"time2ACS";"time2KAST";"time2KD";"time2ADR";"time2KPR";"time2APR";"time2FKPR";"time2FDPR";"time2K";"time2D";"time2A";"time2FK";"time2FD";"ganhador"\r\n"1";789,11;1,014;208,67;71,6;1,04;133,124;0,734;0,292;0,106;0,098;715,772;581,892;164,614;153,286;106,978;694,69;1,01;208,042;70;1,086;135,124;0,732;0,278;0,112;0,102;521,324;456,598;203,22;77,192;64,024;"1"\r\n"2";1350,852;1,024;210,124;71,8;1,078;133,866;0,732;0,278;0,122;0,108;1077,34;904,19;319,998;193,74;148,46;874,128;1,062;219,06;72,2;1,108;139,046;0,772;0,28;0,11;0,098;698,034;612,218;226,66;112,294;96,076;"1"\r\n"3";1152,112;1,05;212,01;73;1,116;136,286;0,746;0,266;0,11;0,094;899,35;757,892;258,592;167,532;125,688;387,302;0,95;205,114;68,4;0,982;132,536;0,718;0,242;0,098;0,112;289,158;283,174;84,748;44,168;43,534;"1"\r\n"4

In [9]:
# Criando variável páginas e criando variável 'p' que será a parte final do url (o número da página)
paginas = []
p = 1

# Criando um laço for que armazenará o url de cada página dentro da variável paginas
for i in range(1, 51):
    paginas.append(f'https://www.vlr.gg/matches/results/?page={p}')
    p += 1

# Variável partidas e variável c
partidas = []
c = 1

# Função que retorna o url de cada partida
def funcaoPagina(pagina):
    response = requests.get(pagina)
    soup = BeautifulSoup(response.text, 'html.parser')
    links = soup.find_all('a')
    partidas = [link.get('href') for link in links]
    partidas = partidas[14:64]
    partidas = ['https://www.vlr.gg' + partida for partida in partidas]
    return partidas


In [10]:
# Criando f e uma lista que receberá todos os returns da funcaoPagina (url de cada partida)
f = 0
a = []

# Executando um for que fará a iteração da funcaoPagina todas as vezes necessárias
for i in paginas:
    a.append(funcaoPagina(paginas[f]))
    f += 1

a = list(itertools.chain(*a))

In [11]:
def medias_Jogadores(url_jogador):
    html_lido = requests.get(url_jogador).text
    soup = BeautifulSoup(html_lido, 'html.parser')
    dados_jogador = []
    for table in soup.find_all('table'):
        dados_jogador.append(table.text)
    dados_jogador = [x.strip().split("\n") for x in dados_jogador]
    dados_jogador = [x for x in dados_jogador if x[0] in ['Use', 'Rating', 'ACS', 'KAST', 'K:D', 'ADR']]
    dados_jogador = [{dados_jogador[i][0]: float(dados_jogador[i][1])} for i in range(len(dados_jogador))]
    dados_jogador = {k: v * dados_jogador[0]['Use'] for d in dados_jogador for k, v in d.items()}
    dados_jogador = {k: round(v / dados_jogador['Use'], 2) for k, v in dados_jogador.items()}
    medias_jogador = dados_jogador
    #medias_jogador['KAST'] = round(medias_jogador['KAST'], 0)
    return medias_jogador


In [12]:
def get_Ganhadores(url_partida):
    html = requests.get(url_partida).text
    soup = BeautifulSoup(html, 'html.parser')
    spoilers = soup.find_all("div", class_="js-spoiler")
    spoilers = [spoiler.text.strip().replace("\t", "").replace("\n", "") for spoiler in spoilers]
    placar = "".join(spoilers)
    match = re.search("(\d+):(\d+)", placar)
    time1, time2 = int(match.group(1)), int(match.group(2))
    ganhador = 1 if time1 > time2 else 0
    return ganhador


In [13]:
def medias_Times(url_partida):
  try:
    # Pegando os dados no link da partida
    html = requests.get(url_partida).text
    soup = BeautifulSoup(html, 'html.parser')
    nodes = soup.select('td.mod-player a')
    links_jogadores = [node['href'] for node in nodes]

    # Separando os nomes dos jogadores de cada time em 2 arrays
    timeA = links_jogadores[:5]
    timeB = links_jogadores[5:]

    # Criando os links usando os nomes dos jogadores
    timeA = ['https://www.vlr.gg/?timespan=all' + i for i in timeA]
    timeB = ['https://www.vlr.gg/?timespan=all' + i for i in timeB]

    timeA_medias = []
    timeB_medias = []

    for i in timeA:
        timeA_medias.append(medias_Jogadores(i))

    for i in timeB:
        timeB_medias.append(medias_Jogadores(i))

    print(timeA_medias)
    print(timeB_medias)
    # timeA_medias = pd.concat(timeA_medias, axis=1)
    # timeB_medias = pd.concat(timeB_medias, axis=1)

    timeA_medias = timeA_medias.drop('Use', axis=1)
    timeB_medias = timeB_medias.drop('Use', axis=1)

    partida = pd.concat([timeA_medias, timeB_medias], axis=1)

    partida.columns = ['time1R', 'time1ACS', 'time1KAST', 'time1KD', 'time1ADR',
                       'time2R', 'time2ACS', 'time2KAST', 'time2KD', 'time2ADR']

    partida = partida[['time1R', 'time2R', 'time1ACS', 'time2ACS', 'time1KAST', 'time2KAST',
                      'time1KD', 'time2KD', 'time1ADR', 'time2ADR']]

    partida = partida.mean(axis=1).transpose()

    ganhador = get_Ganhadores(url_partida)

    partida['ganhador'] = ganhador

    return partida

  except Exception as e:
    print('error:', e)


In [14]:
medias_Times(a[1])

[{}, {}, {}, {}, {}]
[{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}]
error: 'list' object has no attribute 'drop'


In [34]:
# Carregar arquivos de partidas que correspondem aos critérios especificados
padrao = "*_partidas.csv"
arquivos_partidas = glob.glob(padrao)

# Concatenar os dados em um único DataFrame
df = pd.concat([pd.read_csv(arquivo, sep = ';') for arquivo in arquivos_partidas if arquivo.split("_")[0] >= "2023-02-19" and arquivo.split("_")[0] < "2023-04-09"], ignore_index=True)

# Substituir vírgula por ponto em todas as colunas
df = df.replace(',', '.', regex=True)

# Converter todas as colunas para tipo float
df = df.astype('float32')

# Normalizar todas as colunas
scaler = StandardScaler()
df_norm = scaler.fit_transform(df)
df_norm = pd.DataFrame(df_norm, columns=df.columns)

# Selecionar as colunas de entrada e saída
X = df.drop(['ganhador'], axis=1)
y = df['ganhador']

# Converter variável de saída para array de categorias
y = to_categorical(y)

# Dividir os dados em conjuntos de treinamento e teste
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=42)

# Criar modelo de rede neural
modelo = tf.keras.Sequential()
modelo.add(tf.keras.layers.Dense(10, input_dim=X.shape[1], activation='relu'))
modelo.add(tf.keras.layers.Dense(2, activation='softmax'))

# Compilar modelo
modelo.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Treinar modelo
modelo.fit(X_treino, y_treino, epochs=130, batch_size=32, validation_data=(X_teste, y_teste))

# Avaliar modelo
score = modelo.evaluate(X_teste, y_teste, verbose=0)
print(f'Acurácia do modelo: {score[1]}')

Epoch 1/130
17/17 [==============================] - 1s 16ms/step - loss: 574.6169 - accuracy: 0.4637 - val_loss: 482.4485 - val_accuracy: 0.4545
Epoch 2/130
17/17 [==============================] - 0s 5ms/step - loss: 391.5373 - accuracy: 0.4637 - val_loss: 312.1983 - val_accuracy: 0.4545
Epoch 3/130
17/17 [==============================] - 0s 7ms/step - loss: 214.3656 - accuracy: 0.4656 - val_loss: 138.2991 - val_accuracy: 0.5076
Epoch 4/130
17/17 [==============================] - 0s 5ms/step - loss: 102.8052 - accuracy: 0.4695 - val_loss: 107.6818 - val_accuracy: 0.4621
Epoch 5/130
17/17 [==============================] - 0s 4ms/step - loss: 77.9360 - accuracy: 0.5076 - val_loss: 79.6494 - val_accuracy: 0.5303
Epoch 6/130
17/17 [==============================] - 0s 5ms/step - loss: 63.1010 - accuracy: 0.5344 - val_loss: 68.0978 - val_accuracy: 0.5758
Epoch 7/130
17/17 [==============================] - 0s 6ms/step - loss: 54.0232 - accuracy: 0.5458 - val_loss: 57.7842 - val_accurac